In [1]:
import numpy as np
import pandas as pd
from mat4py import loadmat
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.utils import resample

import tensorflow
from tensorflow.keras.callbacks import ModelCheckpoint
import sklearn
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization,Concatenate,concatenate, Input
from tensorflow.keras.layers import Conv2D, Conv1D, MaxPooling1D
#from tensorflow.keras.utils import np_utils
#from tensorflow.keras.utils.vis_utils import model_to_dot
from tensorflow.keras.regularizers import l2

In [2]:
tensorflow.keras.backend.clear_session()

In [3]:
model_ecg = Sequential()
model_ecg.add(BatchNormalization(input_shape=(1408,1)))
model_ecg.add(Conv1D(3, kernel_size=(100),strides=2))
model_ecg.add(Activation("relu"))
model_ecg.add(MaxPooling1D(pool_size=(2),strides=2))
model_ecg.add(Conv1D(50, (10)))
model_ecg.add(MaxPooling1D(pool_size=(2),strides=2))
model_ecg.add(Activation("relu"))
model_ecg.add(Conv1D(30, (30)))
model_ecg.add(MaxPooling1D(pool_size=(2)))
model_ecg.add(Activation("relu"))
model_ecg.add(BatchNormalization())
model_ecg.add(Flatten())
model_ecg.add(Dropout(0.25))
model_ecg.add(Dense(2, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='softmax'))

model_ecg.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 1408, 1)           4         
_________________________________________________________________
conv1d (Conv1D)              (None, 655, 3)            303       
_________________________________________________________________
activation (Activation)      (None, 655, 3)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 327, 3)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 318, 50)           1550      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 159, 50)           0         
_________________________________________________________________
activation_1 (Activation)    (None, 159, 50)           0

In [4]:
model_ecg.load_weights('./best_weights_ecg_32layer.hdf5')

In [5]:
weight_values=model_ecg.get_weights()

In [6]:
first_input = Input(shape=(1408,1))
first_batch = BatchNormalization(weights=weight_values[0:4],trainable=False)(first_input)
first_conv= Conv1D(3,kernel_size=(100),strides=2,weights=[weight_values[4],weight_values[5]],trainable=False)(first_batch)
first_act=Activation("relu")(first_conv)
first_maxpool= MaxPooling1D(pool_size=(2),strides=2)(first_act)
first_conv2=Conv1D(50,kernel_size=(10),weights=[weight_values[6],weight_values[7]],trainable=False)(first_maxpool)
first_maxpool2=MaxPooling1D(pool_size=(2),strides=2)(first_conv2)
first_act2=Activation("relu")(first_maxpool2)
first_conv3=Conv1D(30,kernel_size=(30),weights=[weight_values[8],weight_values[9]],trainable=False)(first_act2)
first_maxpool3=MaxPooling1D(pool_size=(2))(first_conv3)
first_act3=Activation("relu")(first_maxpool3)
first_batch2 = BatchNormalization(weights=weight_values[10:14],trainable=False)(first_act3)
first_final = Flatten()(first_batch2)


In [7]:
model_abdo = Sequential()
model_abdo.add(BatchNormalization(input_shape=(1408,1)))
model_abdo.add(Conv1D(3, kernel_size=(100),strides=2))
model_abdo.add(Activation("relu"))
model_abdo.add(MaxPooling1D(pool_size=(2),strides=2))
model_abdo.add(Conv1D(50, (10)))
model_abdo.add(MaxPooling1D(pool_size=(2),strides=2))
model_abdo.add(Activation("relu"))
model_abdo.add(Conv1D(30, (30)))
model_abdo.add(MaxPooling1D(pool_size=(2)))
model_abdo.add(Activation("relu"))
model_abdo.add(BatchNormalization())
model_abdo.add(Flatten())
model_abdo.add(Dropout(0.25))
model_abdo.add(Dense(2, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='softmax'))

model_abdo.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 1408, 1)           4         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 655, 3)            303       
_________________________________________________________________
activation_6 (Activation)    (None, 655, 3)            0         
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 327, 3)            0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 318, 50)           1550      
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 159, 50)           0         
_________________________________________________________________
activation_7 (Activation)    (None, 159, 50)          

In [8]:
model_abdo.load_weights('./best_weights_abdo_32layer.hdf5')

In [9]:
weight_abdo=model_abdo.get_weights()

In [10]:
third_input = Input(shape=(1408,1))
third_batch = BatchNormalization(weights=weight_abdo[0:4],trainable=False)(third_input)
third_conv= Conv1D(3,kernel_size=(100),strides=2,weights=[weight_abdo[4],weight_abdo[5]],trainable=False)(third_batch)
third_act=Activation("relu")(third_conv)
third_maxpool= MaxPooling1D(pool_size=(2),strides=2)(third_act)
third_conv2=Conv1D(50,kernel_size=(10),weights=[weight_abdo[6],weight_abdo[7]],trainable=False)(third_maxpool)
third_maxpool2=MaxPooling1D(pool_size=(2),strides=2)(third_conv2)
third_act2=Activation("relu")(third_maxpool2)
third_conv3=Conv1D(30,kernel_size=(30),weights=[weight_abdo[8],weight_abdo[9]],trainable=False)(third_act2)
third_maxpool3=MaxPooling1D(pool_size=(2))(third_conv3)
third_act3=Activation("relu")(third_maxpool3)
third_batch2 = BatchNormalization(weights=weight_abdo[10:14],trainable=False)(third_act3)
third_final = Flatten()(third_batch2)


In [11]:
merge_layer=concatenate([first_final, third_final])

In [12]:
merge_batch1 = BatchNormalization()(merge_layer)
merge_drop2 = Dropout(0.25)(merge_batch1)
merge_dense2 = Dense(2, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(merge_drop2)
merge_output= Activation("softmax")(merge_dense2)

In [13]:
model_merge_ecg_abdo = Model(inputs=[first_input, third_input], outputs=merge_output)

In [14]:
model_merge_ecg_abdo.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1408, 1)]    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1408, 1)]    0                                            
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 1408, 1)      4           input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_6 (BatchNor (None, 1408, 1)      4           input_2[0][0]                    
______________________________________________________________________________________________

In [15]:
batch_size = 64
epochs = 500

opt = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)
model_merge_ecg_abdo.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [16]:
list_string=['ucddb002','ucddb003','ucddb005','ucddb006','ucddb007','ucddb009',\
             'ucddb010','ucddb012','ucddb014','ucddb015','ucddb017',\
             'ucddb019','ucddb020','ucddb021','ucddb022','ucddb023','ucddb024',\
             'ucddb025','ucddb026','ucddb027','ucddb028']#'ucddb008','ucddb011','ucddb013','ucddb014',



valid_features2=np.zeros((1,1408))
valid_features3=np.zeros((1,1408))

test_features2=np.zeros((1,1408))
test_features3=np.zeros((1,1408))
valid_labels=np.zeros((1,1))
test_labels=np.zeros((1,1))
for l in list_string:
        
        
        
        ecg_valid = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_ecg_valid.mat')
        ecg_valid = np.array(ecg_valid['ecg_valid'])
        abdo_valid = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_abdo_valid.mat')
        abdo_valid = np.array(abdo_valid['abdo_valid'])
        abdo_valid_labels=loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_valid_labels.mat')
        abdo_valid_labels = np.array(abdo_valid_labels['class_valid'])
        
        valid_features2=np.append(valid_features2,ecg_valid,axis=0)
        valid_features3=np.append(valid_features3,abdo_valid,axis=0)
        valid_labels=np.append(valid_labels,abdo_valid_labels)
       
        
        
        ecg_test = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_ecg_test.mat')
        ecg_test = np.array(ecg_test['ecg_test'])
        abdo_test = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_abdo_test.mat')
        abdo_test = np.array(abdo_test['abdo_test'])
        abdo_test_labels=loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_test_labels.mat')
        abdo_test_labels = np.array(abdo_test_labels['class_test'])
        
        test_features2=np.append(test_features2,ecg_test,axis=0)
        test_features3=np.append(test_features3,abdo_test,axis=0)
        test_labels=np.append(test_labels,abdo_test_labels)
        

ecg_valid=valid_features2[1:]
abdo_valid=valid_features3[1:]
valid_labels=valid_labels[1:]
valid_labels = valid_labels.flatten()


ecg_test=test_features2[1:]
abdo_test=test_features3[1:]
test_labels=test_labels[1:]
test_labels = test_labels.flatten()

In [18]:

train_features2=np.zeros((1,1408))
train_features3=np.zeros((1,1408))

train_labels=np.zeros((1,1))

for l in list_string:
        
        ecg_train = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_ecg_train.mat')
        ecg_train = np.array(ecg_train['ecg_train'])
        abdo_train = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_abdo_train.mat')
        abdo_train = np.array(abdo_train['abdo_train'])
        abdo_train_labels=loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_train_labels.mat')
        abdo_train_labels = np.array(abdo_train_labels['class_train'])
        
        train_features2=np.append(train_features2,ecg_train,axis=0)
        train_features3=np.append(train_features3,abdo_train,axis=0)
        train_labels=np.append(train_labels,abdo_train_labels)
        

ecg_train=train_features2[1:]
abdo_train=train_features3[1:]
train_labels=train_labels[1:]
train_labels = train_labels.flatten()

In [18]:

train_features2=0
train_features3=0

valid_features2=0
valid_features3=0

test_features2=0
test_features3=0

In [19]:
ecg_train_mean=np.mean(ecg_train)
ecg_train_std=np.std(ecg_train)

In [20]:
for i in range(ecg_train.shape[0]):
    ecg_train[i,:]=(ecg_train[i,:]-ecg_train_mean)/ecg_train_std
    
for i in range(ecg_valid.shape[0]):
    ecg_valid[i,:]=(ecg_valid[i,:]-ecg_train_mean)/ecg_train_std
    
for i in range(ecg_test.shape[0]):
    ecg_test[i,:]=(ecg_test[i,:]-ecg_train_mean)/ecg_train_std
    

ecg_train=np.expand_dims(ecg_train, axis=2)
ecg_valid=np.expand_dims(ecg_valid, axis=2)
ecg_test=np.expand_dims(ecg_test, axis=2)

In [21]:
abdo_train_mean=np.mean(abdo_train)
abdo_train_std=np.std(abdo_train)

In [22]:
for i in range(abdo_train.shape[0]):
    abdo_train[i,:]=(abdo_train[i,:]-abdo_train_mean)/abdo_train_std
    
for i in range(abdo_valid.shape[0]):
    abdo_valid[i,:]=(abdo_valid[i,:]-abdo_train_mean)/abdo_train_std
    
for i in range(abdo_test.shape[0]):
    abdo_test[i,:]=(abdo_test[i,:]-abdo_train_mean)/abdo_train_std
    

abdo_train=np.expand_dims(abdo_train, axis=2)
abdo_valid=np.expand_dims(abdo_valid, axis=2)
abdo_test=np.expand_dims(abdo_test, axis=2)

In [23]:
y_train=train_labels
y_valid=valid_labels
y_test=test_labels

In [24]:
num_classes=2
y_train_encoder = sklearn.preprocessing.LabelEncoder()
y_train_num = y_train_encoder.fit_transform(y_train)
y_train_wide = tensorflow.keras.utils.to_categorical(y_train_num, num_classes)

y_valid_num = y_train_encoder.fit_transform(y_valid)
y_valid_wide = tensorflow.keras.utils.to_categorical(y_valid_num, num_classes)

y_test_num = y_train_encoder.fit_transform(y_test)
y_test_wide = tensorflow.keras.utils.to_categorical(y_test_num, num_classes)

In [25]:
best_weights_filepath_merge = './best_weights_fusion_abdo_ecg_14_2.hdf5'
mcp_merge = ModelCheckpoint(best_weights_filepath_merge, monitor="val_accuracy",
                      save_best_only=True, save_weights_only=False)
            
history = model_merge_ecg_abdo.fit([ecg_train,abdo_train], y_train_wide,
         batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([ecg_valid,abdo_valid], y_valid_wide),
          callbacks=[mcp_merge])

Epoch 1/500
12593/12593 [==============================] - 70s 6ms/step - loss: 0.0409 - accuracy: 0.9934 - val_loss: 0.0294 - val_accuracy: 0.9958
Epoch 2/500
12593/12593 [==============================] - 70s 6ms/step - loss: 0.0226 - accuracy: 0.9982 - val_loss: 0.0268 - val_accuracy: 0.9957
Epoch 3/500
12593/12593 [==============================] - 70s 6ms/step - loss: 0.0212 - accuracy: 0.9981 - val_loss: 0.0269 - val_accuracy: 0.9959
Epoch 4/500
12593/12593 [==============================] - 70s 6ms/step - loss: 0.0200 - accuracy: 0.9981 - val_loss: 0.0259 - val_accuracy: 0.9958
Epoch 5/500
12593/12593 [==============================] - 70s 6ms/step - loss: 0.0192 - accuracy: 0.9982 - val_loss: 0.0232 - val_accuracy: 0.9959
Epoch 6/500
12593/12593 [==============================] - 70s 6ms/step - loss: 0.0183 - accuracy: 0.9982 - val_loss: 0.0244 - val_accuracy: 0.9957
Epoch 7/500
12593/12593 [==============================] - 70s 6ms/step - loss: 0.0177 - accuracy: 0.9982 - val_

12593/12593 [==============================] - 71s 6ms/step - loss: 0.0106 - accuracy: 0.9982 - val_loss: 0.0264 - val_accuracy: 0.9933
Epoch 57/500
12593/12593 [==============================] - 71s 6ms/step - loss: 0.0105 - accuracy: 0.9982 - val_loss: 0.0269 - val_accuracy: 0.9954
Epoch 58/500
12593/12593 [==============================] - 73s 6ms/step - loss: 0.0106 - accuracy: 0.9982 - val_loss: 0.0244 - val_accuracy: 0.9949
Epoch 59/500
12593/12593 [==============================] - 71s 6ms/step - loss: 0.0104 - accuracy: 0.9983 - val_loss: 0.0249 - val_accuracy: 0.9945
Epoch 60/500
12593/12593 [==============================] - 72s 6ms/step - loss: 0.0104 - accuracy: 0.9983 - val_loss: 0.0270 - val_accuracy: 0.9952
Epoch 61/500
12593/12593 [==============================] - 71s 6ms/step - loss: 0.0104 - accuracy: 0.9983 - val_loss: 0.0218 - val_accuracy: 0.9952
Epoch 62/500
12593/12593 [==============================] - 71s 6ms/step - loss: 0.0104 - accuracy: 0.9982 - val_loss: 

Epoch 111/500
12593/12593 [==============================] - 70s 6ms/step - loss: 0.0094 - accuracy: 0.9982 - val_loss: 0.0237 - val_accuracy: 0.9957
Epoch 112/500
12593/12593 [==============================] - 71s 6ms/step - loss: 0.0093 - accuracy: 0.9983 - val_loss: 0.0240 - val_accuracy: 0.9932
Epoch 113/500
12593/12593 [==============================] - 70s 6ms/step - loss: 0.0094 - accuracy: 0.9982 - val_loss: 0.0258 - val_accuracy: 0.9948
Epoch 114/500
12593/12593 [==============================] - 70s 6ms/step - loss: 0.0092 - accuracy: 0.9982 - val_loss: 0.0257 - val_accuracy: 0.9922
Epoch 115/500
12593/12593 [==============================] - 70s 6ms/step - loss: 0.0092 - accuracy: 0.9982 - val_loss: 0.0244 - val_accuracy: 0.9948
Epoch 116/500
12593/12593 [==============================] - 70s 6ms/step - loss: 0.0094 - accuracy: 0.9982 - val_loss: 0.0279 - val_accuracy: 0.9938
Epoch 117/500
12593/12593 [==============================] - 70s 6ms/step - loss: 0.0093 - accuracy:

12593/12593 [==============================] - 70s 6ms/step - loss: 0.0086 - accuracy: 0.9981 - val_loss: 0.0293 - val_accuracy: 0.9949
Epoch 220/500
12593/12593 [==============================] - 70s 6ms/step - loss: 0.0086 - accuracy: 0.9982 - val_loss: 0.0332 - val_accuracy: 0.9919
Epoch 221/500
12593/12593 [==============================] - 70s 6ms/step - loss: 0.0086 - accuracy: 0.9981 - val_loss: 0.0254 - val_accuracy: 0.9956
Epoch 222/500
12593/12593 [==============================] - 70s 6ms/step - loss: 0.0088 - accuracy: 0.9981 - val_loss: 0.0281 - val_accuracy: 0.9930
Epoch 223/500
12593/12593 [==============================] - 70s 6ms/step - loss: 0.0088 - accuracy: 0.9981 - val_loss: 0.0324 - val_accuracy: 0.9951
Epoch 224/500
12593/12593 [==============================] - 70s 6ms/step - loss: 0.0087 - accuracy: 0.9981 - val_loss: 0.0255 - val_accuracy: 0.9945
Epoch 225/500
12593/12593 [==============================] - 70s 6ms/step - loss: 0.0086 - accuracy: 0.9982 - val_

Epoch 328/500
12593/12593 [==============================] - 72s 6ms/step - loss: 0.0085 - accuracy: 0.9981 - val_loss: 0.0258 - val_accuracy: 0.9946
Epoch 329/500
12593/12593 [==============================] - 73s 6ms/step - loss: 0.0085 - accuracy: 0.9980 - val_loss: 0.0319 - val_accuracy: 0.9951
Epoch 330/500
12593/12593 [==============================] - 73s 6ms/step - loss: 0.0082 - accuracy: 0.9982 - val_loss: 0.0282 - val_accuracy: 0.9948
Epoch 331/500
12593/12593 [==============================] - 72s 6ms/step - loss: 0.0084 - accuracy: 0.9981 - val_loss: 0.0264 - val_accuracy: 0.9920
Epoch 332/500
12593/12593 [==============================] - 72s 6ms/step - loss: 0.0085 - accuracy: 0.9981 - val_loss: 0.0408 - val_accuracy: 0.9925
Epoch 333/500
12593/12593 [==============================] - 72s 6ms/step - loss: 0.0083 - accuracy: 0.9981 - val_loss: 0.0301 - val_accuracy: 0.9934
Epoch 334/500
12593/12593 [==============================] - 73s 6ms/step - loss: 0.0084 - accuracy:

12593/12593 [==============================] - 75s 6ms/step - loss: 0.0084 - accuracy: 0.9980 - val_loss: 0.0350 - val_accuracy: 0.9937
Epoch 437/500
12593/12593 [==============================] - 75s 6ms/step - loss: 0.0084 - accuracy: 0.9980 - val_loss: 0.0302 - val_accuracy: 0.9911
Epoch 438/500
12593/12593 [==============================] - 75s 6ms/step - loss: 0.0084 - accuracy: 0.9980 - val_loss: 0.0318 - val_accuracy: 0.9943
Epoch 439/500
12593/12593 [==============================] - 75s 6ms/step - loss: 0.0084 - accuracy: 0.9980 - val_loss: 0.0291 - val_accuracy: 0.9952
Epoch 440/500
12593/12593 [==============================] - 75s 6ms/step - loss: 0.0087 - accuracy: 0.9979 - val_loss: 0.0296 - val_accuracy: 0.9922
Epoch 441/500
12593/12593 [==============================] - 75s 6ms/step - loss: 0.0082 - accuracy: 0.9981 - val_loss: 0.0410 - val_accuracy: 0.9907
Epoch 442/500
12593/12593 [==============================] - 76s 6ms/step - loss: 0.0083 - accuracy: 0.9981 - val_

# load weights

In [25]:
model_merge_ecg_abdo.load_weights('./best_weights_fusion_abdo_ecg_14_2.hdf5')

In [26]:
y_pred = model_merge_ecg_abdo.predict([ecg_test,abdo_test])
predict_test=np.argmax(y_pred, axis=1)
predict_test=predict_test.reshape(predict_test.shape[0],1)
cm=confusion_matrix(y_test_num, predict_test)
cm

array([[50246,   122],
       [   15,  1353]], dtype=int64)

In [37]:
(cm[1,1]+cm[0,0])/(cm[1,1]+cm[1,0]+cm[0,0]+cm[0,1])

0.9973519406216175

In [38]:
cm[1,1]/(cm[1,1]+cm[1,0])

0.9890350877192983

In [39]:
cm[0,0]/(cm[0,0]+cm[0,1])

0.9975778271918678